In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from stop_words import get_stop_words
import xgboost as xgb

import pandas as pd
import numpy as np

In [2]:
russian_stop_words = get_stop_words('russian')

In [3]:
#data reading and spliting on test and train
data = pd.read_csv('data/text_BoW_task.csv', encoding='utf-8')


X_train, X_test, y_train, y_test = train_test_split(data['text_stem'],
                                                    data['is_blocked'],
                                                    test_size=0.10,
                                                    random_state=41)

X_train = X_train.astype(unicode);X_test = X_test.astype(unicode)

In [4]:
#creating the bag of word:
vectorizer = CountVectorizer(ngram_range=(1,3))#stop_words=russian_stop_words)
train_data_features = vectorizer.fit_transform(X_train)
test_data_features = vectorizer.transform(X_test)

In [5]:
#creating the bag of word with TF-IDF:
vectorizer_Tfidf = TfidfVectorizer(ngram_range=(1,3))#,stop_words=russian_stop_words)
train_data_features_tfidf = vectorizer_Tfidf.fit_transform(X_train)
test_data_features_tfidf = vectorizer_Tfidf.transform(X_test)

In [ ]:
import xgboost as xgb

sum(train_data_features)

In [7]:
dtrain = xgb.DMatrix(train_data_features,label=y_train)
dtest = xgb.DMatrix(test_data_features,label=y_test)

In [8]:
dtrain_tfidf = xgb.DMatrix(train_data_features_tfidf,label=y_train)
dtest_tfidf = xgb.DMatrix(test_data_features_tfidf,label=y_test)

In [10]:
#no tfidf
param = {'silent':0, 'objective':'binary:logistic', 'booster':'gblinear',
         'alpha' : 0.1, 'lambda': 0.1, 'eta' : 0.25, 'lambda_bias' : 100}

watchlist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 9
bst = xgb.train(param, dtrain, num_round, watchlist)
preds = bst.predict(dtest)
labels = dtest.get_label()
print ('error=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))

[0]	eval-error:0.088856	train-error:0.007447
[1]	eval-error:0.074047	train-error:0.007406
[2]	eval-error:0.068123	train-error:0.007406
[3]	eval-error:0.063680	train-error:0.007406
[4]	eval-error:0.059608	train-error:0.007406
[5]	eval-error:0.057016	train-error:0.007406
[6]	eval-error:0.054424	train-error:0.007406
[7]	eval-error:0.054795	train-error:0.007406
[8]	eval-error:0.054795	train-error:0.007406


error=0.054795


In [211]:
param = {'silent':0, 'objective':'binary:logistic', 'booster':'gblinear',
         'alpha' : 0.01, 'lambda': 0.001, 'eta' : 0.25, 'lambda_bias' : 100}

watchlist  = [(dtest_tfidf,'eval'), (dtrain_tfidf,'train')]
num_round = 12
bst = xgb.train(param, dtrain_tfidf, num_round, watchlist)
preds_tfidf = bst.predict(dtest_tfidf)
labels = dtest_tfidf.get_label()
print ('error=%f' % ( sum(1 for i in range(len(preds_tfidf)) if int(preds_tfidf[i]>0.5)!=labels[i]) /float(len(preds_tfidf))))

[0]	eval-error:0.092558	train-error:0.007406
[1]	eval-error:0.081822	train-error:0.007406
[2]	eval-error:0.071455	train-error:0.007406
[3]	eval-error:0.065902	train-error:0.007406
[4]	eval-error:0.060718	train-error:0.007406
[5]	eval-error:0.058497	train-error:0.007406
[6]	eval-error:0.055905	train-error:0.007406
[7]	eval-error:0.055535	train-error:0.007406
[8]	eval-error:0.054054	train-error:0.007406
[9]	eval-error:0.052203	train-error:0.007406
[10]	eval-error:0.050352	train-error:0.007406
[11]	eval-error:0.049981	train-error:0.007406


error=0.049981


In [170]:
#LogisticRegression training on vectorised features
clf1 = LogisticRegression()
clf1.fit(train_data_features, y_train)
y_pred_1 = clf1.predict(test_data_features)

In [171]:
LogRegression_auc = accuracy_score(y_test, y_pred_1)
LogRegression_accuracy = accuracy_score(y_test, y_pred_1)
print LogRegression_accuracy

0.93817104776


In [172]:
print "xgboost : " + str(accuracy_score(y_test, preds > 0.5))
print "logreg : " + str(LogRegression_accuracy)
print "xgb-tfidf : " + str(accuracy_score(y_test, preds_tfidf > 0.5))

xgboost : 0.945945945946
logreg : 0.93817104776
xgb-tfidf : 0.950018511662


In [242]:
#SVM training on vectorised features
clf2 = LinearSVC()
clf2.fit(train_data_features, y_train)
y_pred_2 = clf2.predict(test_data_features)

In [243]:
SVM_auc = accuracy_score(y_test, y_pred_2)

In [244]:
#creating the bag of word with TF-IDF:
vectorizer_Tfidf = TfidfVectorizer()
train_data_features_tfidf = vectorizer_Tfidf.fit_transform(X_train)
test_data_features_tfidf = vectorizer_Tfidf.transform(X_test)

In [245]:
#LogisticRegression training on vectorised features with TF-IDF transform
clf1_tfidf = LogisticRegression()
clf1_tfidf.fit(train_data_features_tfidf, y_train)
y_pred_1_tfidf = clf1_tfidf.predict(test_data_features_tfidf)

In [246]:
LogRegression_tfidf_auc = accuracy_score(y_test, y_pred_1_tfidf)

In [23]:
#SVM training on vectorised features with TF-IDF transform
clf2_tfidf = LinearSVC()
clf2_tfidf.fit(train_data_features_tfidf, y_train)
y_pred_2_tfidf = clf2.predict(test_data_features_tfidf)

In [24]:
SVM_tfidf_auc = accuracy_score(y_test, y_pred_2_tfidf)

In [25]:
print 'LogRegression_auc: ', LogRegression_auc
print 'SVM_auc: ', SVM_auc
print 'LogRegression_tfidf_auc: ', LogRegression_tfidf_auc
print 'SVM_tfidf_auc: ', SVM_tfidf_auc

LogRegression_auc:  0.935024065161
SVM_auc:  0.92484265087
LogRegression_tfidf_auc:  0.930025916327
SVM_tfidf_auc:  0.91577193632
